# Quick Guide

If you have not yet installed {{ Triumvirate }}, please follow the instructions
in ['Installation'](../installation.rst).

## Using the Python package

### Preamble

Load needed classes and functions from the relevant package modules:

In [1]:
from triumvirate.catalogue import ParticleCatalogue
from triumvirate.logger import setup_logger
from triumvirate.parameters import ParameterSet
from triumvirate.threept import compute_bispec_in_gpp_box

### Optional logger

A logger is useful for program tracking, and can be set up easily and passed
to callables that accept a ``logger`` argument:

In [2]:
trv_logger = setup_logger()

```{seealso}
['Customised Logger'](./Logger.ipynb) for more details.

### Parameter configuration

The simplest way to initialise an instance of the parameter set class
{py:class}`~triumvirate.parameters.ParameterSet` is by loading an external
``.yml`` file. A template parameter file ``"params_template.yml"`` can be
downloaded from [here](../_static/params_template.yml), but the relevant
parameters need to be changed accordingly for your use.

```{seealso}
['Parameter Configuration'](./Configuration.ipynb) and
['Parameter Set'](./Parameters.ipynb) for more details.
```

In [3]:
import yaml

from triumvirate.parameters import fetch_paramset_template

params_dict = fetch_paramset_template('dict')

for ax_name in ['x', 'y', 'z']:
    params_dict['boxsize'][ax_name] = 1000.
    params_dict['ngrid'][ax_name] = 64

params_dict.update({
    'assignment'    : 'cic',
    'catalogue_type': 'survey',
    'statistic_type': 'bispec',
    'degrees'       : {'ell1': 0, 'ell2': 0, 'ELL': 0},
    'range'         : [0.005, 0.105],
    'num_bins'      : 10,
})

with open("params_template.yml", 'w') as params_file:
    yaml.dump(params_dict, params_file, default_flow_style=False)

Assuming the parameter file is in the current working directory, read the
parameter set from file:

In [4]:
parameter_set = ParameterSet("params_template.yml", logger=trv_logger)

[2023-02-26 17:29:23 (+00:00:00) INFO] Validating parameters... (entering C++)
[2023-02-26 17:29:23 (+00:00:00) INFO] ... validated parameters. (exited C++)
[2023-02-26 17:29:23 (+00:00:00) STAT C++] Parameters validated.


The parameters used here are (possibly printed in a different order):

In [5]:
parameter_set.print()

{'alignment': 'centre',
 'assignment': 'cic',
 'binning': 'lin',
 'boxsize': {'x': 1000.0, 'y': 1000.0, 'z': 1000.0},
 'catalogue_type': 'survey',
 'degrees': {'ELL': 0, 'ell1': 0, 'ell2': 0},
 'directories': {'catalogues': None, 'measurements': None},
 'files': {'data_catalogue': None, 'rand_catalogue': None},
 'form': 'diag',
 'idx_bin': None,
 'interlace': 'false',
 'ngrid': {'x': 64, 'y': 64, 'z': 64},
 'norm_convention': 'particle',
 'num_bins': 10,
 'padfactor': None,
 'padscale': 'box',
 'range': [0.005, 0.105],
 'statistic_type': 'bispec',
 'tags': {'output': None},
 'verbose': 20,
 'wa_orders': {'i': None, 'j': None},
 'npoint': '3pt',
 'space': 'fourier'}


### Catalogue data

We will measure clustering statistics from a single simulation-like catalogue
file ``"mock_catalogue_sim.dat"``, which contains three data columns ``'x'``,
``'y'`` and ``'z'`` for the Cartesian coordinates of particles.

In [6]:
import numpy as np
from nbodykit.cosmology import Cosmology

# Cosmology, matter power spectrum and bias at given redshift
cosmo = Cosmology(
    h=0.6736, Omega0_b=0.04930, Omega0_cdm=0.2645, A_s=2.083e-09, n_s=0.9649
)
redshift = 1.
bias = 2.

# Catalogue properties
density = 5.e-4
boxsize = 1000.

# Catalogue selectors
def cut_to_sphere(coords, boxsize):
    return np.less_equal(np.sqrt(np.sum(coords**2, axis=-1)), boxsize/2.)

# Create simulation-like catalogue, or load if existing.
catalogue_sim_filepath = "mock_catalogue_sim.dat"

try:
    catalogue_sim = np.loadtxt(
        catalogue_sim_filepath,
        dtype=[(axis, np.float64) for axis in ['x', 'y', 'z']]
    )
except FileNotFoundError:
    from nbodykit.lab import LinearPower, LogNormalCatalog
    powspec = LinearPower(cosmo, redshift)
    catalogue_sim = LogNormalCatalog(
        powspec, density, boxsize, bias=bias, Nmesh=256, seed=42
    )
    catalogue_sim['Position'] -= boxsize/2.
    np.savetxt(catalogue_sim_filepath, catalogue_sim['Position'].compute())

Read the catalogue data from file:

In [7]:
# DEMO: warning messages are captured and reprinted here for demonstration.

import warnings  # DEMO

with warnings.catch_warnings(record=True) as demo_warnings:
    catalogue = ParticleCatalogue.read_from_file(
        "mock_catalogue_sim.dat",
        names=['x', 'y', 'z'],
        logger=trv_logger
    )
    for demo_warning in demo_warnings:
        print(demo_warning.message)

[2023-02-26 17:29:28 (+00:00:05) INFO] Original extents of particle coordinates: {'x': (-499.998, 499.998), 'y': (-500.000, 499.998), 'z': (-499.999, 500.000)} (ParticleCatalogue(source=extfile:mock_catalogue_sim.dat)).
[2023-02-26 17:29:28 (+00:00:05) INFO] Catalogue loaded: 499214 particles with total sample weight 499214.000 (ParticleCatalogue(source=extfile:mock_catalogue_sim.dat)).
Catalogue 'nz' field is not provided and thus set to zero, which may raise errors in some computations.
Catalogue 'ws' field is not provided, so is set to unity.
Catalogue 'wc' field is not provided, so is set to unity.


```{hint}
As seen in the warning messages, catalogues should usually contain more
data columns than just the Cartesian coordinates. For a cuboid-box catalogue
of unweighted particles like above, the Cartesian coordinates are the only
necessary input data.
```

```{seealso}
['Particle Catalogue'](./Catalogue.ipynb) for more details.
```

### Clustering measurements

As an example, we measure the diagonal global plane-parallel bispectrum
monopole $(\ell_1, \ell_2, L) = (0, 0, 0)$ from the simulation-like cuboid-box
catalogue loaded above, and save the result as a ``.txt`` file to the default
output file ``"bk000_diag.txt"``.

Call the function:

In [8]:
results = compute_bispec_in_gpp_box(
    catalogue, paramset=parameter_set,
    save='.txt', logger=trv_logger
)

[2023-02-26 17:29:28 (+00:00:06) INFO] Validating parameters... (entering C++)
[2023-02-26 17:29:28 (+00:00:06) INFO] ... validated parameters. (exited C++)
[2023-02-26 17:29:28 (+00:00:06) STAT C++] Parameters validated.
[2023-02-26 17:29:28 (+00:00:06) INFO] Parameter set have been initialised.
[2023-02-26 17:29:28 (+00:00:06) INFO] Binning has been initialised.
[2023-02-26 17:29:28 (+00:00:06) INFO] Offset extents of particle coordinates: {'x': (0.000, 1000.000), 'y': (0.000, 999.996), 'z': (0.001, 999.997)} (ParticleCatalogue(source=extfile:mock_catalogue_sim.dat)).
[2023-02-26 17:29:28 (+00:00:06) INFO] Catalogue box has been periodised.
[2023-02-26 17:29:28 (+00:00:06) INFO] Inserted missing 'nz' field based on particle count and boxsize.
[2023-02-26 17:29:28 (+00:00:06) INFO] Preparing catalogue for clustering algorithm... (entering C++)
[2023-02-26 17:29:30 (+00:00:11) INFO C++] Catalogue loaded: 499214 particles with total sample weight 499214.000 (source=extdata).
[2023-02-26

The returned `results` is a dictionary containing the coordinate bin centres
and effective/mean coordinates, the number of contributing modes per bin and
(the real and imaginary parts of) the raw measurements including shot noise and
the shot noise component:

In [9]:
# DEMO
from pprint import pprint
pprint(results)

{'bk_raw': array([-9.76634042e+08+4.68322668e-07j,  5.62308286e+09-2.53143573e-07j,
        8.69371377e+08+1.18607196e-07j,  1.28499842e+09+2.84801617e-09j,
        6.68698395e+08-4.43226461e-09j,  6.05966155e+08+2.08434145e-08j,
        3.85704485e+08+4.52752806e-09j,  2.88772358e+08-1.42805895e-08j,
        2.32705251e+08+7.61606800e-09j,  2.00351320e+08+3.59316825e-09j]),
 'bk_shot': array([1.89812841e+08+2.44859752e-09j, 2.06798892e+08+1.28213346e-09j,
       1.50707290e+08+2.30332332e-10j, 1.24950405e+08-4.50834605e-10j,
       1.06980049e+08-6.02151777e-11j, 9.44867868e+07+1.73202636e-11j,
       8.04644721e+07+6.36183351e-11j, 6.84708228e+07-1.66556058e-10j,
       5.94748920e+07-1.12694819e-10j, 5.12239555e+07+1.19207665e-10j]),
 'k1bin': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ]),
 'k1eff': array([0.01149964, 0.02047114, 0.03052422, 0.04062536, 0.05033379,
       0.06040971, 0.07034279, 0.08025683, 0.09014322, 0.10008191]),
 'k2bin': array([0.01, 0.02,

The saved file contains a header with summary information about the parameters
used and the input data as well as intermediary quantities:

In [10]:
# DEMO
with open("bk000_diag.txt", 'r') as results_file:
    print(results_file.read())

# Catalogue source: extfile:mock_catalogue_sim.dat
# Catalogue size: 499214 particles of total sample weight 499214.000
# Catalogue particle extents: ([0.000, 1000.000], [0.000, 999.996], [0.001, 999.997])
# Box size: (1000.000, 1000.000, 1000.000)
# Box alignment: centre
# Mesh number: (64, 64, 64)
# Mesh assignment and interlacing: cic, false
# Normalisation factor: 8.037846928e+00 (particle-based, used), 2.208590694e+00 (mesh-based, alternative)
# [0] k1_cen, [1] k1_eff, [2] k2_cen, [3] k2_eff, [4] nmodes, [5] Re{bk000_raw}, [6] Im{bk000_raw}, [7] Re{bk000_shot}, [8] Im{bk000_shot}
1.000000000e-02	1.149964290e-02	1.000000000e-02	1.149964290e-02	        56	-9.766340417e+08	 4.683226684e-07	 1.898128415e+08	 2.448597519e-09
2.000000000e-02	2.047114034e-02	2.000000000e-02	2.047114034e-02	       194	 5.623082855e+09	-2.531435728e-07	 2.067988919e+08	 1.282133458e-09
3.000000000e-02	3.052421724e-02	3.000000000e-02	3.052421724e-02	       488	 8.693713775e+08	 1.186071959e-07	 1.507072903e

```{seealso}
['Clustering Measurements'](./Measurements.ipynb) for more details.
```

In [11]:
# Hide cell
!rm params_template.yml bk000_diag.txt

## Running the C++ program

For most users, the C++ code serves as the backend for the Python package
and the only public routine is the compiled binary executable program
``triumvirate`` which should be run with an external parameter file.

### Parameter configuration

A template parameter file ``"params_template.ini"`` can be
downloaded from [here](../_static/params_template.ini), but the relevant
parameters need to be changed accordingly for your use.

```{seealso}
['Parameter Configuration'](./Configuration.ipynb) for more details.
```

In [12]:
import textwrap

params_template = textwrap.dedent(
    """\
    # -- I/O -----------------------------------------------------------------

    # Directories for input/output.
    # The paths can be either absolute or relative to the working directory.
    # If unset, the current working directory is assumed.
    catalogue_dir =
    measurement_dir =

    # Filenames (with extensions) of input/output sources. [mandatory]
    data_catalogue_file = mock_catalogue_sim.dat
    rand_catalogue_file =

    # Column names (comma-separated without space) in catalogue data.
    # [mandatory]
    catalogue_columns = x,y,z

    # Tags to be substituted into input/output paths.
    output_tag =


    # -- Mesh sampling -------------------------------------------------------

    # Box size in each dimension (in Mpc/h). [mandatory]
    boxsize_x = 1000.
    boxsize_y = 1000.
    boxsize_z = 1000.

    # Grid cell number in each dimension. [mandatory]
    ngrid_x = 64
    ngrid_y = 64
    ngrid_z = 64

    # Box alignment: {'centre' (default), 'pad'}.
    alignment = centre

    # Padding scale: {'box' (default), 'grid'}.
    padscale = box

    # Padding factor (as a multiple of the size of padding scale).
    padfactor =

    # Mesh assignment scheme: {'ngp', 'cic', 'tsc' (default), 'pcs'}.
    assignment = tsc

    # Interlacing switch: {'true'/'on', 'false'/'off' (default)}.
    # The switch is overriden to 'false' when measuring three-point statistics.
    interlace = false


    # -- Measurements --------------------------------------------------------

    # Type of catalogue(s): {'survey', 'random', 'sim'}. [mandatory]
    catalogue_type = sim

    # Type of measurement:
    # {'powspec', '2pcf', '2pcf-win', 'bispec', '3pcf', '3pcf-win', '3pcf-win-wa'}.
    # [mandatory]
    statistic_type = bispec

    # Degrees of the multipoles. [optional, optional, mandatory]
    ell1 = 0
    ell2 = 0
    ELL = 0

    # Orders of wide-angle corrections.
    i_wa =
    j_wa =

    # Form of three-point statistic measurements: {'full', 'diag' (default)}.
    form = diag

    # Normalisation convention: {'particle' (default), 'mesh'}.
    norm_convention = particle

    # Binning scheme: {'lin' (default), 'log', 'linpad', 'logpad', 'custom'}.
    binning = lin

    # Minimum and maximum of the range of measurement scales.
    # The binning coordinate is either wavenumbers (in h/Mpc) in Fourier space,
    # or separations (in Mpc/h) in configuration space. [mandatory]
    bin_min = 0.005
    bin_max = 0.105

    # Number of measurement bins (i.e. data vector dimension).
    # Must be >=2, or >=7 if padding is used. [mandatory]
    num_bins = 10

    # Fixed bin index in the full (2-d) three-point statistics measurements.
    idx_bin = 0


    # -- Misc ----------------------------------------------------------------

    # Logging verbosity level: a non-negative integer.
    # Typical values are: {
    #   0 (NSET, unset), 10 (DBUG, debug), 20 (STAT, status) (default),
    #   30 (INFO, info), 40 (WARN, warning), 50 (ERRO, error)
    # }.
    verbose = 20
    """
)

with open("params_template.ini", 'w') as params_template_file:
    params_template_file.write(params_template)

For the example below, we have modified the following lines in the template
parameter file:

```{code-block} ini
:caption: params_template.ini

data_catalogue_file = mock_catalogue_sim.dat

catalogue_columns = x,y,z

boxsize_x = 1000.
boxsize_y = 1000.
boxsize_z = 1000.

ngrid_x = 64
ngrid_y = 64
ngrid_z = 64

catalogue_type = sim

statistic_type = bispec

ell1 = 0
ell2 = 0
ELL = 0

bin_min = 0.005
bin_max = 0.105

num_bins = 10
```

### Executing the program

To run the 'black-box' C++ program, execute
```{code-block} console
$ <path-to-executable> <path-to-parameter-file>
```
in the terminal.

Assuming the compiled binary executable is in the default build directory and
the parameter file is in the current working directory which is the repository
directory root, we execute the following to make the same measurement
as before:
```{code-block} console
$ build/triumvirate params_template.ini
```

In [13]:
%%bash

mkdir -p ./build/
cp ../../../build/triumvirate ./build/triumvirate
build/triumvirate params_template.ini

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
[2023-02-26 17:29:32 (+00:00:00) STAT] [A] Parameters and source data are being initialised.
[2023-02-26 17:29:32 (+00:00:00) STAT] [A.1] Reading parameters...
[2023-02-26 17:29:32 (+00:00:00) STAT] Parameters validated.
[2023-02-26 17:29:32 (+00:00:00) INFO] Check used-parameter file for reference: .//parameters_used.
[2023-02-26 17:29:32 (+00:00:00) STAT] [A.1] ... read parameters.
[2023-02-26 17:29:32 (+00:00:00) STAT] [A.2] Reading catalogues...
[2023-02-26 17:29:32 (+00:00:00) WARN] Catalogue 'nz' field is unavailable and will be set to the mean density in the bounding box (source=extfile:mock_catalogue_sim.dat).
[2023-02-26 17:29:34 (+00:00:02) INFO] Catalogue loaded: 499214 particles with total sample weight 499214.000 (source=extfile:mock_catalogue_sim.dat).
[2023-02-26 17:29:34 (+00:00:02) INFO] Extents of particle coordinates: {'x': (-499.998, 499.998), 'y': (-500.000, 499.998), 'z': (-499.999, 5

As before, the saved file contains a header with summary information about the
parameters used and the input data as well as intermediary quantities:

In [14]:
# DEMO
with open("bk000_diag", 'r') as results_file:
    print(results_file.read())

# Catalogue source: extfile:mock_catalogue_sim.dat
# Catalogue size: 499214 particles of total sample weight 499214.000
# Catalogue particle extents: ([0.000, 1000.000], [0.000, 999.996], [0.001, 999.997])
# Box size: (1000.000, 1000.000, 1000.000)
# Box alignment: centre
# Mesh number: (64, 64, 64)
# Mesh assignment and interlacing: tsc, false
# Normalisation factor: 8.037846928e+00 (particle-based, used), 3.089710869e+00 (mesh-based, alternative)
# [0] k1_cen, [1] k1_eff, [2] k2_cen, [3] k2_eff, [4] nmodes, [5] Re{bk000_raw}, [6] Im{bk000_raw}, [7] Re{bk000_shot}, [8] Im{bk000_shot}
1.000000000e-02	1.149964290e-02	1.000000000e-02	1.149964290e-02	        56	-9.702292027e+08	 0.000000000e+00	 1.898133615e+08	-2.226144357e-26
2.000000000e-02	2.047114034e-02	2.000000000e-02	2.047114034e-02	       194	 5.617152234e+09	 0.000000000e+00	 2.067769130e+08	-7.496980996e-26
3.000000000e-02	3.052421724e-02	3.000000000e-02	3.052421724e-02	       488	 8.716164555e+08	 0.000000000e+00	 1.506636764e

In addition, the interpreted/processed parameters used by the program are
saved to the ``"parameters_used*"`` file in the specified measurement
directory (the current working directory here), where the wildcard asterisk
is the output tag (empty here) specified in the input parameter file.

In [15]:
# DEMO
with open("parameters_used", 'r') as used_params_file:
    print(used_params_file.read())

catalogue_dir = 
measurement_dir = ./
data_catalogue_file = mock_catalogue_sim.dat
rand_catalogue_file = 
catalogue_columns = x,y,z
output_tag = 
boxsize_x = 1000.00
boxsize_y = 1000.00
boxsize_z = 1000.00
ngrid_x = 64
ngrid_y = 64
ngrid_z = 64
volume = 1.000000e+09
nmesh = 262144
alignment = centre
padscale = box
padfactor = 0.0000
assignment = tsc
interlace = false
catalogue_type = sim
statistic_type = bispec
npoint = 3pt
space = fourier
ell1 = 0
ell2 = 0
ELL = 0
i_wa = 0
j_wa = 0
form = diag
norm_convention = particle
binning = lin
bin_min = 0.0050
bin_max = 0.1050
num_bins = 10
idx_bin = 0
verbose = 20



In [16]:
# Hide cell.
!rm -r ./build/ ./params_template.ini ./parameters_used ./bk000_diag